In [ ]:
import cvxpy as cp
import numpy as np
from itertools import combinations
from gcspy import GraphOfConvexPrograms

In [2]:
# initialize empty graph
graph = GraphOfConvexPrograms()

# vertex 1
v1 = graph.add_vertex("v1")
x1 = v1.add_variable(2)
c1 = np.array([-3, 0]) # center of the set
v1.add_constraint(cp.norm_inf(x1 - c1) <= 1)

# vertex 2
v2 = graph.add_vertex("v2")
x2 = v2.add_variable(2)
c2 = np.array([0, 2.5]) # center of the set
D2 = np.diag([.25, 2]) # scaling matrix
v2.add_constraint(cp.norm_inf(D2 @ (x2 - c2)) <= 1)

# vertex 3
v3 = graph.add_vertex("v3")
x3 = v3.add_variable(2)
c3 = np.array([3, 0]) # center of the set
v3.add_constraint(cp.norm2(x3 - c3) <= 1)
v3.add_constraint(x3 >= c3) # keep only top right part of the set

# vertex 4
v4 = graph.add_vertex("v4")
x4 = v4.add_variable(2)
c4 = np.array([0, -2.5]) # center of the set
D4 = np.diag([1, 2]) # scaling matrix
v4.add_constraint(cp.norm2(D4 @ (x4 - c4)) <= 1)

# vertex 5
v5 = graph.add_vertex("v5")
x5 = v5.add_variable(2)
c5 = np.array([.3, .3]) # center of the set
v5.add_constraint(cp.norm1(x5 - c5) <= 1)

In [3]:
# add an edges vetween every pair of distinct vertices
for tail in graph.vertices:
    for head in graph.vertices:
        if tail != head:
            edge = graph.add_edge(tail, head)

            # edge cost is Euclidean distance
            edge.add_cost(cp.norm2(tail.variables[0] - head.variables[0]))

In [4]:
# solve traveling salesman problem using Dantzig–Fulkerson–Johnson formulation
prob = graph.solve_traveling_salesman()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

Problem status: optimal
Optimal value: 12.807842389466414


In [5]:
# if the method solve_traveling_salesman was not implemented, we could still
# solve the TSP by passing the constraints of its integer linear program (ILP)
# formulation (in this case we write again the DFJ formulation)

# the lower bound of 0 and upper bound of 1 on the binary variables are
# automatically enforced and do not have to be included in the formulation

# binary variables
yv = graph.vertex_binaries()
ye = graph.edge_binaries()

# vertex constraints
ilp_constraints = []
for i, vertex in enumerate(graph.vertices):
    inc = graph.incoming_edge_indices(vertex)
    out = graph.outgoing_edge_indices(vertex)
    ilp_constraints += [yv[i] == 1, sum(ye[out]) == 1, sum(ye[inc]) == 1]

# subtour elimnation constraints
for subtour_size in range(2, graph.num_vertices() - 1):
    for vertices in combinations(graph.vertices, subtour_size):
        out = graph.outgoing_edge_indices(vertices)
        ilp_constraints.append(sum(ye[out]) >= 1)

In [6]:
# solve traveling salesman problem from constraints of the ilp formulation and
# check that optimal value is equal to the one above
prob = graph.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

Problem status: optimal
Optimal value: 12.807842389466414
